## **Running the models using the 'modelling' package**

A notebook through which different modelling configurations can be ran, using the ``modelling`` package. It follows the steps of:
- preparing packages;
- setting "global" variables;
- getting the data;
- defining hyperparameters;
- running a grid search and/or training a model; and
- evaluation.
In the modelling package, variations can be made to the models and training functions to experiment. Don't forget to restart the notebook after making changes there.

For future models, a suggestion is to embed the training/testing functions in a Model class, instead of keeping them loose from each other. (With, optimally, inheritance from a base class, etc etc, such that there is minimal code duplication.) This way, the training procedure can be easily tailored per model. In the current set-up, different functions have to be called for fully-connected networks and hierarchical networks because they handle the data differently. Another way this would be a worth investment, is for implementation of physics-informed models, which require a whole physics injection into the training procedure. In that case, tight coupling is much recommended over the current state of this file. Therefore, I'd first change the code such that it works per model and such that only functionalities independent of model type are actually independent/loosely coupled from the models, therewith facilitating scalable experimentation.

Throughout the notebook, there are printing statements to clarify potential errors happening on Habrok

In [1]:
print("Starting script...")

from modelling import *
from modelling import GRU
from modelling import HGRU
from modelling.plots import set_minmax_path, set_contaminants

import os
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset

Starting script...

Running __init__.py for data pipeline...
Modelling package initialized



Use GPU when available

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

Device:  cuda


### **Set "global" variables**

In [3]:
HABROK = bool(0)                  # set to True if using HABROK; it will print
                                  # all stdout to a .txt file to log progress
CITY_NAME = "Utrecht"
BASE_DIR = Path.cwd()
MODEL_PATH = BASE_DIR / "results" / "models"
MINMAX_PATH = BASE_DIR.parent / "data" / "data_combined" / CITY_NAME.lower() / "contaminant_minmax.csv"

print("BASE_DIR: ", BASE_DIR)
print("MODEL_PATH: ", MODEL_PATH)
print("MINMAX_PATH: ", MINMAX_PATH)

torch.manual_seed(34)             # set seed for reproducibility

N_HOURS_U = 72                    # number of hours to use for input
N_HOURS_Y = 24                    # number of hours to predict
N_HOURS_STEP = 24                 # "sampling rate" in hours of the data; e.g. 24 
                                  # means sample an I/O-pair every 24 hours
                                  # the contaminants and meteorological vars
CONTAMINANTS = ['NO2', 'O3'] # 'PM10', 'PM25']
COMPONENTS = ['NO2', 'O3', 'PM10', 'PM25', 'SQ', 'WD', 'Wvh', 'dewP', 'p', 'temp']

BASE_DIR:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/src
MODEL_PATH:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/src/results/models
MINMAX_PATH:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/data/data_combined/utrecht/contaminant_minmax.csv


### **Load in data and create PyTorch *Datasets***

In [4]:
# Load in data and create PyTorch Datasets. To tune
# which exact .csv files get extracted, change the
# lists in the get_dataframes() definition

train_input_frames = get_dataframes('train', 'u', CITY_NAME)
train_output_frames = get_dataframes('train', 'y', CITY_NAME)

val_input_frames = get_dataframes('val', 'u', CITY_NAME)
val_output_frames = get_dataframes('val', 'y', CITY_NAME)

test_input_frames = get_dataframes('test', 'u', CITY_NAME)
test_output_frames = get_dataframes('test', 'y', CITY_NAME)

print("Successfully loaded data")

Successfully loaded data


In [5]:
train_dataset = TimeSeriesDataset(
    train_input_frames,  # list of input training dataframes
    train_output_frames, # list of output training dataframes
    5,                   # number of dataframes put in for both
                         # (basically len(train_input_frames) and
                         # len(train_output_frames) must be equal)
    N_HOURS_U,           # number of hours of input data
    N_HOURS_Y,           # number of hours of output data
    N_HOURS_STEP,        # number of hours between each input/output pair
)
val_dataset = TimeSeriesDataset(
    val_input_frames,    # etc.
    val_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)
test_dataset = TimeSeriesDataset(
    test_input_frames,
    test_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)

del train_input_frames, train_output_frames
del val_input_frames, val_output_frames
del test_input_frames, test_output_frames

### **Define hyperparameters**

In [10]:
# Here, all (hyper)parameters are defined. The hyperparameters are defined in
# a dictionary, which is then passed to the model and the training functions.
# The grid search is performed by generating all possible combinations of the
# hyperparameters defined in the hp_space dictionary, and then performing k-fold cross
# validation on each of these configurations. The best configuration is then returned.
# When the search is finished, comment out the hp_space dictionary and save the best found
# hyperparameters in the hp dictionary, and train the final model with these.

hp = {
    'n_hours_u' : N_HOURS_U,
    'n_hours_y' : N_HOURS_Y,

    'model_class' : GRU, # changed to GRU
    'input_units' : 8, #train_dataset.__n_features_in__(),
    'hidden_layers' : 4,
    'hidden_units' : 64,
    # 'branches' : 2,  # predicting only no2 and o3
    'output_units' : 2, #train_dataset.__n_features_out__(),

    'Optimizer' : torch.optim.Adam,
    'lr_shared' : 1e-3,
    'scheduler' : torch.optim.lr_scheduler.ReduceLROnPlateau,
    'scheduler_kwargs' : {'mode' : 'min',
                          'factor' : 0.1,
                          'patience' : 3,
                          'cooldown' : 8,
                          'verbose' : True},
    'w_decay' : 1e-7,
    'loss_fn' : torch.nn.MSELoss(),

    'epochs' : 5000,
    'early_stopper' : EarlyStopper,
    'patience' : 20,
    'batch_sz' : 16,
    'k_folds' : 5,
}                                   # The lr for the branched layer(s) is calculated
                                    # based on the "power ratio" between the branched
                                    # part of the network and the shared layer, which
                                    # is *assumed* to be proportional to n_hidden_layers
# hp['lr_branch'] = hp['lr_shared'] * hp['hidden_layers']

hp_space = {
    'n_hours_u': N_HOURS_U,  # Try different input time windows
    'n_hours_y': N_HOURS_Y,  # Different forecast horizons
    
    'model_class': GRU,  # If you want to test multiple models, add them here
    'input_units': [8],  # Number of input features (could match your dataset size)
    'hidden_layers': [2, 4, 6],  # Number of hidden layers to test
    'hidden_units': [32, 64, 128],  # Size of hidden units per layer
    'output_units': [2],  # Fixed at 2 since you’re predicting NO2 and O3
    
    'Optimizer': [torch.optim.Adam],  # You could add others like SGD, RMSprop, etc.
    'lr_shared': [1e-3, 1e-4, 5e-4],  # Learning rates to test
    'scheduler': [torch.optim.lr_scheduler.ReduceLROnPlateau],
    'scheduler_kwargs': [
        {'mode': 'min', 'factor': 0.1, 'patience': 3, 'cooldown': 8, 'verbose': True},
        {'mode': 'min', 'factor': 0.5, 'patience': 5, 'cooldown': 10, 'verbose': True}
    ],
    'w_decay': [1e-7, 1e-6, 1e-5],  # Weight decay values
    
    'loss_fn': [nn.MSELoss()],
    
    'epochs': [1000, 3000, 5000],  # Different max epochs for longer/shorter training
    'early_stopper': [EarlyStopper],
    'patience': [10, 20, 30],  # Patience for early stopping
    'batch_sz': [16, 32, 64],  # Mini-batch sizes
    'k_folds': [5],  # Number of folds for cross-validation
}


### **Start hyperparameter search/training**

In [11]:
# print("starting training...")

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
stdout_location = f'results/grid_search_exe_s/exe_of_GRU_at_{current_time}.txt'
train_dataset_full = ConcatDataset([train_dataset, val_dataset])
#                                     # If HABROK, print to external file, else print to stdout
with PrintManager(stdout_location, 'a', HABROK):
    print(f"Grid search execution of GRU at {current_time}\n")
                                    # Train on the full training set
    model, best_hp, val_loss = grid_search(hp, hp_space, train_dataset_full, hier=False, verbose=True)
                                    # Externally save the best model
    torch.save(model.state_dict(), f"{MODEL_PATH}/model_GRU.pth")

    hp = update_dict(hp, best_hp)   # Update the hp dictionary with the best hyperparameters
    print_dict_vertically(best_hp)

Grid search execution of GRU at 20250223-123853

CURRENT CONFIGURATION:
n_hours_u       : 72
n_hours_y       : 24
model_class     : <class 'modelling.GRU.GRU'>
input_units     : 8
hidden_layers   : 2
hidden_units    : 32
output_units    : 2
Optimizer       : <class 'torch.optim.adam.Adam'>
lr_shared       : 0.001
scheduler       : <class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
scheduler_kwargs: {'mode': 'min', 'factor': 0.1, 'patience': 3, 'cooldown': 8, 'verbose': True}
w_decay         : 1e-07
loss_fn         : MSELoss()
epochs          : 1000
early_stopper   : <class 'modelling.EarlyStopper.EarlyStopper'>
patience        : 10
batch_sz        : 16
k_folds         : 5


	Fold 1/5
Epoch: 1 	Ltrain: 0.040494 	Lval: 0.021509
Epoch: 5 	Ltrain: 0.016046 	Lval: 0.015338
Epoch: 10 	Ltrain: 0.011361 	Lval: 0.010846
Epoch: 15 	Ltrain: 0.010357 	Lval: 0.009620
Epoch: 20 	Ltrain: 0.009066 	Lval: 0.009063
Epoch: 25 	Ltrain: 0.008425 	Lval: 0.008408
Epoch: 30 	Ltrain: 0.007674 	Lval: 0.008015

KeyboardInterrupt: 

Lay out model architecture with optimal hyperparameters

In [22]:
hp_gru = {
    'n_hours_u' : N_HOURS_U,
    'n_hours_y' : N_HOURS_Y,

    'model_class' : GRU, # changed to GRU
    'input_units' : 8, #train_dataset.__n_features_in__(),
    'hidden_layers' : 4,
    'hidden_units' : 128,
    # 'branches' : 2,  # predicting only no2 and o3
    'output_units' : 2, #train_dataset.__n_features_out__(),

    'Optimizer' : torch.optim.Adam,
    'lr_shared' : 1e-3,
    'scheduler' : torch.optim.lr_scheduler.ReduceLROnPlateau,
    'scheduler_kwargs' : {'mode' : 'min',
                          'factor' : 0.1,
                          'patience' : 3,
                          'cooldown' : 8,
                          'verbose' : True},
    'w_decay' : 1e-5,
    'loss_fn' : torch.nn.MSELoss(),

    'epochs' : 5000,
    'early_stopper' : EarlyStopper,
    'patience' : 15,
    'batch_sz' : 16,
    'k_folds' : 5,
}

In [23]:

with PrintManager(stdout_location, 'a', HABROK):
    print("\nPrinting model:")
    model_final = GRU(hp['n_hours_u'],
                 hp_gru['n_hours_y'],
                 hp_gru['input_units'],
                 hp_gru['hidden_layers'],
                 hp_gru['hidden_units'], 
                #  hp['branches'],
                 hp_gru['output_units'])
    print(model_final)


Printing model:
GRU(
  (gru): GRU(8, 128, num_layers=4, batch_first=True)
  (dense): Linear(in_features=128, out_features=2, bias=True)
)


Train model on complete training dataset (= train + validation)

In [24]:
train_loader = DataLoader(train_dataset, batch_size = hp_gru['batch_sz'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = hp_gru['batch_sz'], shuffle = False) 
                                            
#                                         # Train the final model on the full training set,
#                                         # save the final model, and save the losses for plotting
with PrintManager(stdout_location, 'a', HABROK):
    print("\nTraining on full training set...")
    model_final, train_losses, val_losses = \
        train(hp_gru, train_loader, val_loader, True)
    torch.save(model_final.state_dict(), f'{MODEL_PATH}/model_GRU.pth')

df_losses = pd.DataFrame({'L_train': train_losses, 'L_val': val_losses})
df_losses.to_csv(f'{os.path.join(os.getcwd(), "results/final_losses")}/losses_HGRU_at_{current_time}.csv', 
                 sep = ';', decimal = '.', encoding = 'utf-8')


Training on full training set...


Epoch: 1 	Ltrain: 0.018103 	Lval: 0.010405
Epoch: 5 	Ltrain: 0.006252 	Lval: 0.005842
Epoch: 10 	Ltrain: 0.005726 	Lval: 0.005462
Epoch: 15 	Ltrain: 0.005309 	Lval: 0.004166
Epoch 00016: reducing learning rate of group 0 to 1.0000e-04.
Epoch: 20 	Ltrain: 0.005038 	Lval: 0.004245
Epoch: 25 	Ltrain: 0.005032 	Lval: 0.003877
Epoch 00029: reducing learning rate of group 0 to 1.0000e-05.
Epoch: 30 	Ltrain: 0.005005 	Lval: 0.004012
Epoch: 35 	Ltrain: 0.004999 	Lval: 0.004051
Epoch: 40 	Ltrain: 0.005005 	Lval: 0.004037
EarlyStopper: stopping at epoch 39 with best_val_loss = 0.003877



#### **Testing the model**

In [25]:

model_final.load_state_dict(torch.load(f"{MODEL_PATH}/model_GRU.pth"))
print(model_final)

GRU(
  (gru): GRU(8, 128, num_layers=4, batch_first=True)
  (dense): Linear(in_features=128, out_features=2, bias=True)
)


In [26]:
def test(model, loss_fn, test_loader, denorm=False, path=None) -> float:
    model.eval()
    test_loss = np.float64(0)
    
    # Ensure the model is on the correct device
    device = next(model.parameters()).device

    with torch.no_grad():
        for batch_test_u, batch_test_y in test_loader:
            batch_test_u = batch_test_u.to(device)
            batch_test_y = batch_test_y.to(device)
            
            pred = model(batch_test_u)
            if denorm:
                pred = denormalise(pred, path)
                batch_test_y = denormalise(batch_test_y, path)
            
            test_loss += loss_fn(pred, batch_test_y).item()

    return test_loss / len(test_loader)


In [27]:
test_loader = DataLoader(test_dataset, batch_size = hp['batch_sz'], shuffle = False) 
loss_fn = nn.MSELoss()  # Instantiate the loss function
test_error = test(model_final, loss_fn, test_loader)

with PrintManager(stdout_location, 'a', HABROK):
    print()
    print("Testing MSE:", test_error)


Testing MSE: 0.003313543042168021


In [28]:
def test_separately(
    model,
    loss_fn,
    test_loader,
    denorm: bool = False,
    path: str = None,
    components=["NO2", "O3", "PM10", "PM25"],
):
    """
    Evaluates on test set and returns test loss

    :param model: model to evaluate, must be some PyTorch type model
    :param loss_fn: loss function to use, PyTorch defined, or PyTorch inherited
    :param test_loader: DataLoader to get batches from
    :param denorm: whether to denormalise the data before calculating loss
    :param path: path to the file containing the minmax values for the data
    :return: dictionary with contaminant names as keys and losses as values
    """
    model.eval()
    device = next(model.parameters()).device
    test_losses = [np.float64(0) for _ in components]

    with torch.no_grad():
        for batch_test_u, batch_test_y in test_loader:
            pred = model(batch_test_u.to(device))
            if denorm:
                pred = denormalise(pred, path)
                batch_test_y = denormalise(batch_test_y.to(device), path)

            for comp in range(len(components)):
                test_losses[comp] += loss_fn(
                    pred[:, :, comp], batch_test_y[:, :, comp]
                ).item()

    for comp in range(len(components)):
        test_losses[comp] /= len(test_loader)
    return {comp: loss for comp, loss in zip(components, test_losses)}


In [29]:
print(test(model_final, nn.MSELoss(), train_loader))
print(test(model_final, nn.MSELoss(), val_loader))
print(test(model_final, nn.MSELoss(), test_loader))

print("\nMSE Training set:")
print_dict_vertically(
    test_separately(model_final, nn.MSELoss(), train_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)
print("\nMSE Validation set:")
print_dict_vertically(
    test_separately(model_final, nn.MSELoss(), val_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)
print("\nMSE Test set:")
print_dict_vertically(
    test_separately(model_final, nn.MSELoss(), test_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)

0.005013745170222913
0.004090381398176153
0.003313543042168021

MSE Training set:
NO2: 86.59752915545208
O3 : 85.67562345179115

MSE Validation set:
NO2: 63.03963724772135
O3 : 91.78997039794922

MSE Test set:
NO2: 52.24487431844076
O3 : 70.96932665506999


In [30]:
print("\nRMSE Training set:")
print_dict_vertically_root(
    test_separately(model_final, nn.MSELoss(), train_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)
print("\nRMSE Validation set:")
print_dict_vertically_root(
    test_separately(model_final, nn.MSELoss(), val_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)
print("\nRMSE Test set:")
print_dict_vertically_root(
    test_separately(model_final, nn.MSELoss(), test_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)


RMSE Training set:
NO2: 9.305779356645862
O3 : 9.256112731246459

RMSE Validation set:
NO2: 7.939750452484093
O3 : 9.580708240936534

RMSE Test set:
NO2: 7.2280615878975985
O3 : 8.424329448393504


In [31]:
pair = 2

set_minmax_path(MINMAX_PATH)
set_contaminants(CONTAMINANTS)

plot_pred_vs_gt(model_final, test_dataset, pair, 'NO2', N_HOURS_Y)
plot_pred_vs_gt(model_final, test_dataset, pair, 'O3', N_HOURS_Y)

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at cuda:0